# Tarea 1 - Amazon food review





En primer lugar se carga el corpus, parseando todo código HTML existente con BeautifulSoup. Así solo se mantendrá el contenido que exista en dentro de los tag. Las reseñas se almacenan en la lista **corpus**, la cual tiene una estructura de lista de listas. Corpus[[documento1],[documento2],...]. A su vez, se tiene una lista **score** con la evaluación de los usuarios en cada reseña (esta información será utilizada más adelante).

In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup

reviews = open("amazon-fine-foods/Reviews.csv", "r")
corpus = []
score = []

reviews.readline() # sacar header

for line in reviews:
    review = line.strip().split(",")
    text = BeautifulSoup(review[-1], 'html.parser')
    corpus.append([text.get_text()])
    score.append(review[6])

reviews.close()

Una vez cargada la información del corpus se procede a tokenizar, manteniendo las palabras sin puntuación alguna. Además se agrega la expresión regular para capturar valores númericos como **3.88**. Esta información se guarda en la lista **tokens_doc** (lista de listas de tokens). Los signos de puntuación también quedan dentro de la lista y serán eliminados en el siguiente paso.

In [2]:
from nltk.tokenize import RegexpTokenizer, regexp_tokenize

tokens_doc = [] #tokens por documento/sentencia
#tokenizer = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)

for text in corpus:
    #t = tokenizer.tokenize(text[0])
    t = regexp_tokenize(text[0], pattern='\w+|\$[\d.]+|\S')
    tokens_doc.append(t)
print tokens_doc[:2]

[[u'I', u'have', u'bought', u'several', u'of', u'the', u'Vitality', u'canned', u'dog', u'food', u'products', u'and', u'have', u'found', u'them', u'all', u'to', u'be', u'of', u'good', u'quality', u'.', u'The', u'product', u'looks', u'more', u'like', u'a', u'stew', u'than', u'a', u'processed', u'meat', u'and', u'it', u'smells', u'better', u'.', u'My', u'Labrador', u'is', u'finicky', u'and', u'she', u'appreciates', u'this', u'product', u'better', u'than', u'most', u'.'], [u'"', u'Product', u'arrived', u'labeled', u'as', u'Jumbo', u'Salted', u'Peanuts', u'.', u'.', u'.', u'the', u'peanuts', u'were', u'actually', u'small', u'sized', u'unsalted', u'.', u'Not', u'sure', u'if', u'this', u'was', u'an', u'error', u'or', u'if', u'the', u'vendor', u'intended', u'to', u'represent', u'the', u'product', u'as', u'"', u'"', u'Jumbo', u'"', u'"', u'.', u'"']]


Una vez que tenemos los tokens para cada documento, se carga la lista de stopwords y se procede a filtrar las palabras de cada documento. Los criterios para que no sean eliminados son:

* No estar en la stoplist
* Largo mayor o igual a 3
* Frecuencia de la palabra en el corpus completo sea mayor o igual a 4

En este proceso también se aplica *lower case a las palabras*.

Los tokens son guardados en la lista tokens, lista de lista de tokens.

In [3]:
import nltk
from nltk.corpus import stopwords

all_tokens = [token.lower() for doc in tokens_doc for token in doc]

stoplist = stopwords.words('english') # stopwords
frec_dist = nltk.FreqDist(all_tokens) # frec distribution of tokens

# create list of list without stopwords
tokens = []
for doc in tokens_doc: 
    t = []
    
    for token in doc:
        low = token.lower()
        if low not in stoplist and len(low) >= 3 and frec_dist[low] >= 4:
            t.append(low)
    tokens.append(t)
            
print tokens[:2]

[[u'bought', u'several', u'vitality', u'canned', u'dog', u'food', u'products', u'found', u'good', u'quality', u'product', u'looks', u'like', u'stew', u'processed', u'meat', u'smells', u'better', u'labrador', u'finicky', u'appreciates', u'product', u'better'], [u'product', u'arrived', u'labeled', u'jumbo', u'salted', u'peanuts', u'peanuts', u'actually', u'small', u'sized', u'unsalted', u'sure', u'error', u'vendor', u'intended', u'represent', u'product', u'jumbo']]


Collocations

In [4]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

finder = BigramCollocationFinder.from_words(all_tokens)
top_collocations = finder.nbest(bigram_measures.pmi, 30) #top 30 bigramas

print top_collocations[:3]

[(u'$500.00', u'deductable'), (u'01014', u'c196132y112a'), (u'04830', u'powerpop')]


In [5]:
print top_collocations[-3:]

[(u'425mgsugar', u'20gcalcium'), (u'45mgpotassium', u'425mgsugar'), (u'4gfiber', u'0gsugar')]


In [8]:
'45mgpotassium' in corpus

False

In [9]:
from nltk.tag import StanfordPOSTagger

jar = 'pos-tagger/stanford-postagger.jar'
model = 'pos-tagger/english-bidirectional-distsim.tagger'


st = StanfordPOSTagger(model,jar)
print tokens[0]
print st.tag(tokens[0])

pos = [st.tag(doc) for doc in tokens[:5000]]

print "pos"
print pos[:2]

[u'bought', u'several', u'vitality', u'canned', u'dog', u'food', u'products', u'found', u'good', u'quality', u'product', u'looks', u'like', u'stew', u'processed', u'meat', u'smells', u'better', u'labrador', u'finicky', u'appreciates', u'product', u'better']
[(u'bought', u'VBD'), (u'several', u'JJ'), (u'vitality', u'NN'), (u'canned', u'VBD'), (u'dog', u'NN'), (u'food', u'NN'), (u'products', u'NNS'), (u'found', u'VBD'), (u'good', u'JJ'), (u'quality', u'NN'), (u'product', u'NN'), (u'looks', u'VBZ'), (u'like', u'IN'), (u'stew', u'NN'), (u'processed', u'JJ'), (u'meat', u'NN'), (u'smells', u'VBZ'), (u'better', u'JJR'), (u'labrador', u'NN'), (u'finicky', u'NN'), (u'appreciates', u'VBZ'), (u'product', u'NN'), (u'better', u'RBR')]
pos
[[(u'bought', u'VBD'), (u'several', u'JJ'), (u'vitality', u'NN'), (u'canned', u'VBD'), (u'dog', u'NN'), (u'food', u'NN'), (u'products', u'NNS'), (u'found', u'VBD'), (u'good', u'JJ'), (u'quality', u'NN'), (u'product', u'NN'), (u'looks', u'VBZ'), (u'like', u'IN'), (

In [ ]:
from nltk.tag import StanfordNERTagger

jar = 'ner-tagger/stanford-ner.jar'
model = 'ner-tagger/english.all.3class.distsim.crf.ser.gz'


st = StanfordNERTagger(model, jar) 
ner = [st.tag(doc) for doc in tokens]

sentiment analysis

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
sid.polarity_scores(tokens[0]) #probar